In [211]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [99]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210225
dia = 25
mes = 02
año = 2021
25/02/2021


In [100]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [101]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

## Importa BBDD ASECORP

In [102]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [103]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [104]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [105]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [106]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [107]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [108]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [109]:
# titulo

In [110]:
len(titulo)

2959

In [111]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [112]:
boletin[0:25]

['[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [113]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [114]:
n_vacios

276

In [115]:
vacios[0:25]

['Acuerdo Multilateral RID 1/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Mercancías Peligrosas por Ferrocarril (RID), relativo a los certificados de consejeros de seguridad de conformidad con el punto 1.8.3.7 del RID, hecho en Madrid el 25 de marzo de 2020',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Acuerdo Multilateral RID 2/2020 en virtud de la Sección 1.5.1 del Reglamento del Transporte Internacional de Merc

In [116]:
boletin[0:25]

[['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268'],
 ['Acuerdo Multilateral M-271'],
 ['Acuerdo Multilateral M-273'],
 ['Acuerdo Multilateral M-276'],
 ['Acuerdo Multilateral M-291'],
 ['Acuerdo Multilateral M-292']]

In [117]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [118]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268',
 'Acuerdo Multilateral M-271',
 'Acuerdo Multilateral M-273',
 'Acuerdo Multilateral M-276',
 'Acuerdo Multilateral M-291',
 'Acuerdo Multilateral M-292']

In [119]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [120]:
boletin_ASECORP_flat_list[0:25]

['Ley 38/1995',
 'Circular 1/2013',
 'Orden de 30 de junio de 1998',
 'Nota de servicio 1/2014',
 'Orden JUS/206/2009',
 'Real Decreto 2100/1976',
 'Orden FOM/3459/2003',
 'Orden PRE/985/2007',
 'Real Decreto 399/2013',
 'Orden de 20 de mayo de 2016',
 'Orden SND/445/2020',
 'Resolución de 10 de enero de 2020',
 'Real decreto 1976/1999',
 'Real Decreto-ley 9/2000',
 'Real Decreto 1075/2017',
 'Real Decreto 1326/1995',
 'Orden de 2 de abril de 1993',
 'Resolución de 16 de noviembre de 2010',
 'Orden de 13 de junio de 1984',
 'Orden SCO/1591/2005',
 'Real Decreto 984/2015',
 'Orden FOM/882/2015',
 'Resolución de 24 de septiembre de 2012',
 'Resolución de 8 de marzo de 2012',
 'Orden MAM/1873/2004']

In [121]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

# ------------------------------------------------------------------------

# DOGC

In [122]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [123]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8350']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ', 'Anuncios varios']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Administración_de_justicia_', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_varios']


In [124]:
#response

## Recoge Nombre Secciones Sumario

In [125]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [126]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia
5,Anuncios varios


## Recoge Valores para formar URLs Secciones Sumario

In [127]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_de_la_Administración_de_justicia_
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_varios


In [128]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
5,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [129]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [130]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
5,Anuncios varios,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [131]:
df_sumarios['URL_Seccion'][5]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Anuncios_varios'

## Recoge Items en Seccion Disposiciones

In [132]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [133]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [134]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Disposiciones


In [135]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [136]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8350#section-Disposiciones'

In [137]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Vicepresidencia y de Economía y Hacienda', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Trabajo, Asuntos Sociales y Familias']
['RESOLUCIÓN VEH/481/2021, de 8 de febrero, por la que se da publicidad al Convenio de colaboración entre la Agencia Tributaria de Cataluña y la Agencia de la Vivienda de Cataluña para la recaudación en período ejecutivo de ingresos tributarios y otros ingresos de derecho público.', 'EDICTO de 17 de febrero de 2021, sobre un acuerdo de la Comisión de Territorio de Cataluña referente a un Plan especial urbanístico del municipio de Alguaire.', "EDICTO de 18 de febrero de 2021, sobre un acuerdo de la Comisión Territorial de Urbanismo de L'Alt Pirineu referente al municipio de Puigcerdà.", 'EDICTO de 18 de febrero de 2021, sobre un acuerdo de la Comisión Territorial de Urbanismo de Girona referente al municipio de Campdevànol.', 'EDICTO de 18 de febrero de 2021, sobre un acuerdo de la Comisión de Te

In [138]:
len(departamentos)

4

In [139]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [140]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [141]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [142]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"RESOLUCIÓN VEH/481/2021, de 8 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"EDICTO de 17 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,CORRECCIÓN DE ERRATA en el Edicto de 29 de oct...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"RESOLUCIÓN TES/479/2021, de 22 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"ACUERDO GOV/19/2021, de 23 de febrero, por el ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,"RESOLUCIÓN TSF/480/2021, de 23 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [143]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=894173'

In [191]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [192]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,tipo_doc,fecha_doc,numero_doc,num_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,numero_control,item_urlXML,item_title,item_fullText
0,"RESOLUCIÓN VEH/481/2021, de 8 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Resolución],[08/02/2021],[VEH/0481/2021],,[Agencia Tributaria de Cataluña],[CVE-DOGC-B-21054033-2021],[8350],[25/02/2021],[Disposiciones],[21054033],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[RESOLUCIÓN VEH/481/2021, de 8 de febrero, por...",En fecha 17 de diciembre de 2020 se ha firmado...
1,"EDICTO de 17 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[17/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049037-2021],[8350],[25/02/2021],[Disposiciones],[21049037],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 17 de febrero de 2021, sobre un acu...","La Comisión de Territorio de Cataluña, en la s..."
2,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049024-2021],[8350],[25/02/2021],[Disposiciones],[21049024],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...",La Comisión Territorial de Urbanismo del Alto ...
3,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049071-2021],[8350],[25/02/2021],[Disposiciones],[21049071],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...",La Comisión Territorial de Urbanismo de Girona...
4,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049046-2021],[8350],[25/02/2021],[Disposiciones],[21049046],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...","La Comisión de Territorio de Cataluña, en la s..."
5,CORRECCIÓN DE ERRATA en el Edicto de 29 de oct...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Corrección de erratas],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049076-2021],[8350],[25/02/2021],[Disposiciones],[21049076],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[CORRECCIÓN DE ERRATA en el Edicto de 29 de oc...,Exp. 2020/072181/M\nPlan especial urbanístico ...
6,"RESOLUCIÓN TES/479/2021, de 22 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Resolución],[22/02/2021],[TES/0479/2021],,[Agencia Catalana del Agua],[CVE-DOGC-B-21054035-2021],[8350],[25/02/2021],[Disposiciones],[21054035],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[RESOLUCIÓN TES/479/2021, de 22 de febrero, po...",\nEn cumplimiento de lo establecido en el art...
7,"ACUERDO GOV/19/2021, de 23 de febrero, por el ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Acuerdo del gobierno ],[23/02/2021],[GOV/0019/2021],,[Departamento de Cultura],[CVE-DOGC-B-21055025-2021],[8350],[25/02/2021],[Disposiciones],[21055025],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[ACUERDO GOV/19/2021, de 23 de febrero, por el...","Por la Resolución CLT/677/2020, de 4 de marzo ..."
8,"RESOLUCIÓN TSF/480/2021, de 23 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Resolución]

In [197]:
DOGC_sumarios['item_fullText'][0]

'En fecha 17 de diciembre de 2020 se ha firmado el Convenio de colaboración entre la Agencia Tributaria de Cataluña y la Agencia de la Vivienda de Cataluña para la recaudación en período ejecutivo de ingresos tributarios y otros ingresos de derecho público.\nVistos el artículo 112.2 de la Ley 26/2010, de 3 de agosto, de régimen jurídico y de procedimiento de las administraciones públicas de Cataluña, y el artículo 11 de la Ley 40/2015, de 1 de octubre, de Régimen Jurídico del Sector Público,\n\xa0\nResuelvo:\n\xa0\nHacer público el Convenio de colaboración entre la Agencia Tributaria de Cataluña y la Agencia de la Vivienda de Cataluña para la recaudación en período ejecutivo de ingresos tributarios y otros ingresos de derecho público.\n \xa0\nBarcelona, 8 de febrero de 2021\n\xa0\nEduard Vilà Marhuenda\nDirector de la Agencia Tributaria de Cataluña\n\xa0\n\xa0\nCONVENIO\nConvenio de colaboración entre la Agencia Tributaria de Cataluña y la Agencia de la Vivienda de Cataluña para la rec

In [218]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [221]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [222]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Ley 26/2010', 'Ley 40/2015', 'Ley 17/2017', 'Ley 13/2009', 'Ley 17/2017', 'Ley 13/2009', 'Ley 13/2009', 'Ley 18/2007', 'Ley 18/2007', 'Real Decreto 939/2005', 'Orden VEH/117/2017', 'Ley 58/2003', 'Ley 39/2015', 'Ley 40/2015', 'Reglamento (UE) 2016/679', 'Ley 26/2010', 'Ley 17/2017', 'Real Decreto 939/2005', 'Ley 13/2009', 'Ley 18/2007', 'Real Decreto 939/2005', 'Ley 58/2003', 'Ley 26/2010']
['Ley 5/2020', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 5/2020', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
[]
['Ley 26/2010', 'Ley 10/2011']
['Resolución CLT/677/2020', 'Ley 9/1993', 'Ley 9/1993']
['Ley 6/2006', 'Ley 33/2010', 'Resolución TSF/961/2020', 'Resolución TSF/961/2020', 'Real decreto 887/2006', 'Ley 4/1985', 'Ley 4/2020',

In [223]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Ley 17/2017', 'Orden VEH/117/2017', 'Ley 26/2010', 'Ley 39/2015', 'Ley 13/2009', 'Ley 58/2003', 'Reglamento (UE) 2016/679', 'Ley 18/2007', 'Real Decreto 939/2005', 'Ley 40/2015']
['Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012', 'Ley 5/2020', 'Ley 29/1998']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007']
['Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012', 'Ley 5/2020', 'Ley 29/1998']
[]
['Ley 26/2010', 'Ley 10/2011']
['Resolución CLT/677/2020', 'Ley 9/1993']
['Ley 33/2010', 'Ley 4/1985', 'Ley 4/2020', 'Ley 26/2010', 'Ley 39/2015', 'Resolución TSF/961/2020', 'Ley 6/2006', 'Real decreto 887/2006']


In [224]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [225]:
print(regex_result)

['Ley 17/2017', 'Orden VEH/117/2017', 'Ley 26/2010', 'Ley 39/2015', 'Ley 13/2009', 'Ley 58/2003', 'Reglamento (UE) 2016/679', 'Ley 18/2007', 'Real Decreto 939/2005', 'Ley 40/2015', 'Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012', 'Ley 5/2020', 'Ley 29/1998', 'Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007', 'Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 2/2007', 'Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012', 'Ley 5/2020', 'Ley 29/1998', 'Ley 26/2010', 'Ley 10/2011', 'Resolución CLT/677/2020', 'Ley 9/1993', 'Ley 33/2010', 'Ley 4/1985', 'Ley 4/2020', 'Ley 26/2010', 'Ley 39/2015', 'Resolución TSF/961/2020', 'Ley 6/2006', 'Real decreto 887/2006']


In [226]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [227]:
boletin_flat_list

['Ley 33/2010',
 'Ley 10/2011',
 'Ley 9/1993',
 'Ley 5/2020',
 'Ley 29/1998',
 'Ley 13/2009',
 'Real decreto 887/2006',
 'Ley 17/2017',
 'Ley 4/2020',
 'Ley 26/2010',
 'Resolución CLT/677/2020',
 'Ley 3/2012',
 'Resolución TSF/961/2020',
 'Ley 6/2006',
 'Reglamento (UE) 2016/679',
 'Ley 40/2015',
 'Orden VEH/117/2017',
 'Ley 4/1985',
 'Ley 2/2007',
 'Ley 39/2015',
 'Ley 58/2003',
 'Ley 18/2007',
 'Real Decreto 939/2005']

In [212]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


['España', 'Europa', 'Cataluña']


In [228]:
boletin_ASECORP_flat_list

"['Ley 38/1995', 'Circular 1/2013', 'Orden de 30 de junio de 1998', 'Nota de servicio 1/2014', 'Orden JUS/206/2009', 'Real Decreto 2100/1976', 'Orden FOM/3459/2003', 'Orden PRE/985/2007', 'Real Decreto 399/2013', 'Orden de 20 de mayo de 2016', 'Orden SND/445/2020', 'Orden de 27 de junio de 2000', 'Resolución de 10 de enero de 2020', 'Real decreto 1976/1999', 'Real Decreto-ley 9/2000', 'Real Decreto 1075/2017', 'Real Decreto 1326/1995', 'Resolución MAH/3481/2008', 'Orden de 2 de abril de 1993', 'Resolución de 16 de noviembre de 2010', 'Orden de 13 de junio de 1984', 'Orden SCO/1591/2005', 'Real Decreto 984/2015', 'Orden FOM/882/2015', 'Resolución de 24 de septiembre de 2012', 'Resolución de 21 de marzo de 1989', 'Ley 26/2009', 'Resolución de 8 de marzo de 2012', 'Orden MAM/1873/2004', 'Orden de 16 de enero de 1995', 'Orden IET/697/2015', 'Orden ITC/3190/2011', 'Real Decreto 60/2005', 'Resolución INT/2927/2011', 'Orden ECO/4/2016', 'Resolución MAH/3589/2008', 'Orden de 27 de julio de 199

In [229]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [230]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 33/2010'}


In [231]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 33/2010'} LEY 33/2010 ESP


In [232]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,tipo_doc,fecha_doc,numero_doc,num_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,numero_control,item_urlXML,item_title,item_fullText,Tags,Match_ASECORP_BBDD
0,"RESOLUCIÓN VEH/481/2021, de 8 de febrero, por ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Resolución],[08/02/2021],[VEH/0481/2021],,[Agencia Tributaria de Cataluña],[CVE-DOGC-B-21054033-2021],[8350],[25/02/2021],[Disposiciones],[21054033],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[RESOLUCIÓN VEH/481/2021, de 8 de febrero, por...",En fecha 17 de diciembre de 2020 se ha firmado...,"[Ley 17/2017, Orden VEH/117/2017, Ley 26/2010,...",[]
1,"EDICTO de 17 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[17/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049037-2021],[8350],[25/02/2021],[Disposiciones],[21049037],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 17 de febrero de 2021, sobre un acu...","La Comisión de Territorio de Cataluña, en la s...","[Ley 2/2007, Ley 39/2015, Ley 3/2012, Ley 5/20...",[LEY 3/2012 ESP]
2,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049024-2021],[8350],[25/02/2021],[Disposiciones],[21049024],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...",La Comisión Territorial de Urbanismo del Alto ...,"[Ley 3/2012, Ley 39/2015, Ley 29/1998, Ley 2/2...",[LEY 3/2012 ESP]
3,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049071-2021],[8350],[25/02/2021],[Disposiciones],[21049071],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...",La Comisión Territorial de Urbanismo de Girona...,"[Ley 3/2012, Ley 39/2015, Ley 29/1998, Ley 2/2...",[LEY 3/2012 ESP]
4,"EDICTO de 18 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Edicto ],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049046-2021],[8350],[25/02/2021],[Disposiciones],[21049046],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[EDICTO de 18 de febrero de 2021, sobre un acu...","La Comisión de Territorio de Cataluña, en la s...","[Ley 2/2007, Ley 39/2015, Ley 3/2012, Ley 5/20...",[LEY 3/2012 ESP]
5,CORRECCIÓN DE ERRATA en el Edicto de 29 de oct...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Corrección de erratas],[18/02/2021],[],,[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21049076-2021],[8350],[25/02/2021],[Disposiciones],[21049076],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[CORRECCIÓN DE ERRATA en el Edicto de 29 de oc...,Exp. 2020/072181/M\nPlan especial urbanístico ...,[],[]
6,"RESOLUCIÓN TES/479/2021, de 22 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Resolución],[22/02/2021],[TES/0479/2021],,[Agencia Catalana del Agua],[CVE-DOGC-B-21054035-2021],[8350],[25/02/2021],[Disposiciones],[21054035],[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,"[RESOLUCIÓN TES/479/2021, de 22 de febrero, po...",\nEn cumplimiento de lo establecido en el art...,"[Ley 26/2010, Ley 10/2011]",[]
7,"ACUERDO GOV/19/2021, de 23 de febrero, por el ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,[Acuerdo del gobierno ],[23/02/2021],[GOV/0019/2021],,[Departamento de Cultura],[

# Genera Fichero EXCEL de resultados

In [251]:
DOGC_sumarios_final = DOGC_sumarios

In [252]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
tipo_doc              object
fecha_doc             object
numero_doc            object
num_control           object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
numero_control        object
item_urlXML           object
item_title            object
item_fullText         object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [253]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [254]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [255]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [256]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [259]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [260]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=894230&type=01&language=es_ES";"CVE-DOGC-B-21054033-2021")'

In [263]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [264]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21054033-2021,"RESOLUCIÓN VEH/481/2021, de 8 de febrero, por ...",25/02/2021,"Ley 17/2017, Orden VEH/117/2017, Ley 26/2010, ...",
1,CVE-DOGC-B-21049037-2021,"EDICTO de 17 de febrero de 2021, sobre un acue...",25/02/2021,"Ley 2/2007, Ley 39/2015, Ley 3/2012, Ley 5/202...",LEY 3/2012 ESP
2,CVE-DOGC-B-21049024-2021,"EDICTO de 18 de febrero de 2021, sobre un acue...",25/02/2021,"Ley 3/2012, Ley 39/2015, Ley 29/1998, Ley 2/2007",LEY 3/2012 ESP
3,CVE-DOGC-B-21049071-2021,"EDICTO de 18 de febrero de 2021, sobre un acue...",25/02/2021,"Ley 3/2012, Ley 39/2015, Ley 29/1998, Ley 2/2007",LEY 3/2012 ESP
4,CVE-DOGC-B-21049046-2021,"EDICTO de 18 de febrero de 2021, sobre un acue...",25/02/2021,"Ley 2/2007, Ley 39/2015, Ley 3/2012, Ley 5/202...",LEY 3/2012 ESP
5,CVE-DOGC-B-21049076-2021,CORRECCIÓN DE ERRATA en el Edicto de 29 de oct...,25/02/2021,,
6,CVE-DOGC-B-21054035-2021,"RESOLUCIÓN TES/479/2021, de 22 de febrero, por...",25/02/2021,"Ley 26/2010, Ley 10/2011",
7,CVE-DOGC-B-21055025-2021,"ACUERDO GOV/19/2021, de 23 de febrero, por el ...",25/02/2021,"Resolución CLT/677/2020, Ley 9/1993",
8,CVE-DOGC-B-21054067-2021,"RESOLUCIÓN TSF/480/2021, de 23 de febrero, por...",25/02/2021,"Ley 33/2010, Ley 4/1985, Ley 4/2020, Ley 26/20...",LEY 33/2010 ESP


In [265]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 